### Generate Data --- SORT

In [1]:
import os, random, json, re
import pandas as pd
import numpy as np
from tqdm import trange, tqdm
import numpy as np
TASK = "sort"

In [28]:
samples_per_num_digit = 1000
data = {}
for num_digit in trange(129, 257):
    unique_sorted_strings = set()
    strings = []
    while len(strings) < samples_per_num_digit:
        partition = sorted(random.sample(list(range(1, num_digit+10)), 9))
        prev = 0
        buckets = []
        for b in partition:
            buckets.append(b-prev-1)
            prev = b
        buckets.append(num_digit+10-prev-1)
        s = "".join([str(d)*n for n, d in zip(buckets, list(range(10)))])
        
        if s not in unique_sorted_strings:
            unique_sorted_strings.add(s)
            strings.append("".join(random.sample(s, len(s))))

    data[num_digit] = strings
json.dump(data, open(f"../data/finetune/{TASK}/finetune_129_256.json", "w"), indent=2)
            

100%|██████████| 128/128 [00:07<00:00, 16.32it/s]


In [27]:
# Uniform split
samples_per_num_digit_train, samples_per_num_digit_val = 100, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        row = {
            "input_str": [s],
            "answer": ["".join(sorted(s))]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        row = {
            "input_str": [s],
            "answer": ["".join(sorted(s))]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/uniform_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/uniform_split/val.csv", index=False)

#val.to_csv(f"../data/finetune/{TASK}/uniform_large_split/val.csv", index=False)


samples_per_num_digit = 2000


  0%|          | 0/124 [00:00<?, ?it/s]

100%|██████████| 124/124 [00:06<00:00, 17.81it/s]


12400 12400


In [30]:
# length_extrapolation split
samples_per_num_digit_train, samples_per_num_digit_val = 0, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune_129_256.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        row = {
            "input_str": [s],
            "answer": ["".join(sorted(s))]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        row = {
            "input_str": [s],
            "answer": ["".join(sorted(s))]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

val.to_csv(f"../data/finetune/{TASK}/length_extrapolation/val.csv", index=False)

samples_per_num_digit = 1000


100%|██████████| 128/128 [00:03<00:00, 33.64it/s]


0 12800


In [2]:
# Random labels
samples_per_num_digit_train, samples_per_num_digit_val = 100, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        row = {
            "input_str": [s],
            "answer": ["".join([random.choice("0123456789") for i in range(int(num_digit))])]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/random_labels/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/random_labels/val.csv", index=False)

#val.to_csv(f"../data/finetune/{TASK}/uniform_large_split/val.csv", index=False)


samples_per_num_digit = 2000


  0%|          | 0/124 [00:00<?, ?it/s]

100%|██████████| 124/124 [00:05<00:00, 21.30it/s]


12400 12400


### Draft

In [2]:
# odd_even_3:1 split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 150
    else: samples_per_num_digit_train = 50
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_even_3:1_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_even_3:1_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:06<00:00, 18.37it/s]


12600 12600


In [3]:
# odd_even_9:1 split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 180
    else: samples_per_num_digit_train = 20
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_even_9:1_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_even_9:1_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:06<00:00, 18.19it/s]


12600 12600


In [4]:
# odd_only split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 100
    else: samples_per_num_digit_train = 0
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_only_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_only_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:05<00:00, 25.01it/s]

6300 12600


### Compute ACC

In [1]:
from collections import Counter
import json, re, math, sys
from pprint import pprint
import numpy as np
from tqdm import tqdm
sys.path.append("../scripts/")
from helper import *

In [3]:
with open("../scripts/llama/output/1206_121802/eval_samples/1206_122016.txt", "r") as f:
    lines = f.readlines()
em = 0
lcs_errors = []
all_preds = []
for l in tqdm(lines):
    gth, pred = json.loads(l)
    gth_result = int(gth.split(".")[0])
    pred_result = -1
    find = re.findall(r'(\d+)', pred)
    if find: pred_result = int(find[0])
    all_preds.append(pred_result)
    em += int(gth_result == pred_result) # int(-10<=gth_len-pred_len<=10) #
    if not int(gth_result == pred_result): lcs_errors.append(len(str(gth_result)) - lcs(str(gth_result), str(pred_result)))
print(f"testing acc = {em / len(lines)}") 
print(f"avg lcs_error = ", np.mean(lcs_errors))

  0%|          | 0/12400 [00:00<?, ?it/s]

100%|██████████| 12400/12400 [00:14<00:00, 845.96it/s] 

testing acc = 0.00024193548387096774
avg lcs_error =  3.7843833185448092
